This is a notebook to try out the standard Voltronic RS232 protocol.

## Required Libraries

| Name       | Description                                            |
|------------|--------------------------------------------------------|
| `pyserial` | For serial communication with the inverter controller. |
| `pandas`   | For displaying fancy tables.                           |

In [1]:
import serial
import pandas as pd

## Checksum

The Voltronic protocol uses CRC-16 (XMODEM variant) for calculating the checksum for messages.

In [2]:
CRC16_XMODEM_TABLE = [
    0x0000, 0x1021, 0x2042, 0x3063, 0x4084, 0x50a5, 0x60c6, 0x70e7,
    0x8108, 0x9129, 0xa14a, 0xb16b, 0xc18c, 0xd1ad, 0xe1ce, 0xf1ef,
    0x1231, 0x0210, 0x3273, 0x2252, 0x52b5, 0x4294, 0x72f7, 0x62d6,
    0x9339, 0x8318, 0xb37b, 0xa35a, 0xd3bd, 0xc39c, 0xf3ff, 0xe3de,
    0x2462, 0x3443, 0x0420, 0x1401, 0x64e6, 0x74c7, 0x44a4, 0x5485,
    0xa56a, 0xb54b, 0x8528, 0x9509, 0xe5ee, 0xf5cf, 0xc5ac, 0xd58d,
    0x3653, 0x2672, 0x1611, 0x0630, 0x76d7, 0x66f6, 0x5695, 0x46b4,
    0xb75b, 0xa77a, 0x9719, 0x8738, 0xf7df, 0xe7fe, 0xd79d, 0xc7bc,
    0x48c4, 0x58e5, 0x6886, 0x78a7, 0x0840, 0x1861, 0x2802, 0x3823,
    0xc9cc, 0xd9ed, 0xe98e, 0xf9af, 0x8948, 0x9969, 0xa90a, 0xb92b,
    0x5af5, 0x4ad4, 0x7ab7, 0x6a96, 0x1a71, 0x0a50, 0x3a33, 0x2a12,
    0xdbfd, 0xcbdc, 0xfbbf, 0xeb9e, 0x9b79, 0x8b58, 0xbb3b, 0xab1a,
    0x6ca6, 0x7c87, 0x4ce4, 0x5cc5, 0x2c22, 0x3c03, 0x0c60, 0x1c41,
    0xedae, 0xfd8f, 0xcdec, 0xddcd, 0xad2a, 0xbd0b, 0x8d68, 0x9d49,
    0x7e97, 0x6eb6, 0x5ed5, 0x4ef4, 0x3e13, 0x2e32, 0x1e51, 0x0e70,
    0xff9f, 0xefbe, 0xdfdd, 0xcffc, 0xbf1b, 0xaf3a, 0x9f59, 0x8f78,
    0x9188, 0x81a9, 0xb1ca, 0xa1eb, 0xd10c, 0xc12d, 0xf14e, 0xe16f,
    0x1080, 0x00a1, 0x30c2, 0x20e3, 0x5004, 0x4025, 0x7046, 0x6067,
    0x83b9, 0x9398, 0xa3fb, 0xb3da, 0xc33d, 0xd31c, 0xe37f, 0xf35e,
    0x02b1, 0x1290, 0x22f3, 0x32d2, 0x4235, 0x5214, 0x6277, 0x7256,
    0xb5ea, 0xa5cb, 0x95a8, 0x8589, 0xf56e, 0xe54f, 0xd52c, 0xc50d,
    0x34e2, 0x24c3, 0x14a0, 0x0481, 0x7466, 0x6447, 0x5424, 0x4405,
    0xa7db, 0xb7fa, 0x8799, 0x97b8, 0xe75f, 0xf77e, 0xc71d, 0xd73c,
    0x26d3, 0x36f2, 0x0691, 0x16b0, 0x6657, 0x7676, 0x4615, 0x5634,
    0xd94c, 0xc96d, 0xf90e, 0xe92f, 0x99c8, 0x89e9, 0xb98a, 0xa9ab,
    0x5844, 0x4865, 0x7806, 0x6827, 0x18c0, 0x08e1, 0x3882, 0x28a3,
    0xcb7d, 0xdb5c, 0xeb3f, 0xfb1e, 0x8bf9, 0x9bd8, 0xabbb, 0xbb9a,
    0x4a75, 0x5a54, 0x6a37, 0x7a16, 0x0af1, 0x1ad0, 0x2ab3, 0x3a92,
    0xfd2e, 0xed0f, 0xdd6c, 0xcd4d, 0xbdaa, 0xad8b, 0x9de8, 0x8dc9,
    0x7c26, 0x6c07, 0x5c64, 0x4c45, 0x3ca2, 0x2c83, 0x1ce0, 0x0cc1,
    0xef1f, 0xff3e, 0xcf5d, 0xdf7c, 0xaf9b, 0xbfba, 0x8fd9, 0x9ff8,
    0x6e17, 0x7e36, 0x4e55, 0x5e74, 0x2e93, 0x3eb2, 0x0ed1, 0x1ef0,
]

def crc16_xmodem(data: bytes) -> int:
    crc = 0

    for byte in data:
        index = ((crc >> 8) ^ byte) & 0xFF
        crc = (CRC16_XMODEM_TABLE[index] ^ (crc << 8)) & 0xFFFF

    return crc & 0xFFFF

In [3]:
hex(crc16_xmodem(bytes("QPI", "ascii")))
# Expected output: 0xbeac

'0xbeac'

In [4]:
def pack_message(message: str, encoding='ascii') -> bytes:
    """
    Packs the message into the transmission ready format.
    
    Calculates the checksum of the message and appends it to the message with a carriage return
    as specified by the protocol.
    """
    result = bytearray(message, encoding)

    checksum = crc16_xmodem(bytes(result))
    
    result.append(checksum >> 8)
    result.append(checksum & 0xFF)
    result.append(0x0D) # Carriage Return

    return bytes(result)

("QPI", pack_message("QPI").hex(' '))

('QPI', '51 50 49 be ac 0d')

In [5]:
def unpack_message(data: bytes, encoding='ascii') -> str:
    """
    Unpack the message from the transmitted format.

    Validates the checksum and strips it from the message as specified by the protocol.
    """

    if data[-1] != 0x0D:
        print(f'Data: {data.decode("ascii", errors="backslashreplace")}')
        raise Exception("Message missing carriage return.")
    raw_message = data[:-3]
    
    packed_checksum = (data[-3] << 8) | data[-2]
    calculated_checksum = crc16_xmodem(raw_message)

    if packed_checksum != calculated_checksum:
        print(f'Data: {data.decode("ascii", errors="backslashreplace")}')
        print(f'Calculate Checksum: {calculated_checksum}')
        print(f'   Packed Checksum: {packed_checksum}')
        raise Exception("Unmatching checksum.")
    
    if raw_message[0] == 0x28: # '(' character.
        raw_message = raw_message[1:]

    return raw_message.decode(encoding)

unpack_message(pack_message('QPI'))

'QPI'

In [6]:
!python3 -m serial.tools.list_ports -v

/dev/ttyUSB0        
    desc: USB-Serial Controller C
    hwid: USB VID:PID=067B:2303 LOCATION=1-1.5
1 ports found


In [7]:
connection = serial.Serial('/dev/ttyUSB0', baudrate=2400, timeout=2)

In [8]:
def execute_command(command: str, reset_buffers=False) -> str:
    if reset_buffers:
        connection.reset_input_buffer()
        connection.reset_output_buffer()

    connection.write(pack_message(command))
    return unpack_message(connection.read_until(b'\r'))

### Query Protocol ID

In [9]:
execute_command('QPI')

'PI30'

### Query Device Serial Number

In [10]:
execute_command('QID')

'<208q008<20808'

### Query MPPT CPU Firmware Version

In [11]:
execute_command('QVFW')

'VERFW:10270.03'

### Query Model Name

In [12]:
execute_command('QMN')

'VMII-NXPW5KW'

### Query Device Mode

| Code  | Mode              |
|-------:|-------------------|
| P     | Power On Mode     |
| S     | Standby Mode      |
| Y     | Bypass Mode       |
| **L** | **Line Mode**     |
| **B** | **Battery Mode**  |
| BT    | Battery Test Mode |
| F     | Fault Mode        |
| D     | Shutdown Mode     |
| G     | Grid Mode         |
| **C** | **Charge Mode**   |

In [13]:
execute_command('QMOD')

'L'

### Query Device Rated Information

_Details are available in the official documents_

In [14]:
qpiri_data = execute_command('QPIRI').split(' ')

In [15]:
pd.DataFrame(zip(qpiri_data, [
    'Grid Rating Voltage (V)',
    'Gird Rating Current (A)',
    'AC Output Rating Voltage (V)',
    'AC Output Rating Frequency (Hz)',
    'AC Output Rating Current (A)',
    'AC Output Rating Appearent Power (W)',
    'AC Output Rating Active Power (W)',
    'Battery Rating Voltage (V)',
    'Battery Re-charge Voltage (V)',
    'Battery Under Voltage (V)',
    'Battery Bulk Voltage (V)',
    'Battery Float Voltage (V)',
    'Battery Type (AGM, Flooded, User)',
    'AC Charing Current (A)',
    'Current Max Charging Current (A)',
    'Input Voltage Range (0: Appliance, 1: UPS)',
    'Output Source (0: Utility, 1: Solar)',
    'Charge Source (1: Solar, 2: Solar & Utility, 3: Only Solar)',
    'Parallel Maximum Number',
    'Machine Type (00: Grid tie, 01,30: Standalone, 10: Hybrid)',
    'Topology (0: Tranformer-less, 1: Transformer)',
    'Output Mode',
    'Battery Re-discharge Voltage (V)',
    'PV OK',
    'PV Power Balance',
]), columns=['Value', 'Description'])

,Value,Description
0,220.0,Grid Rating Voltage (V)
1,13.0,Gird Rating Current (A)
2,220.0,AC Output Rating Voltage (V)
3,50.0,AC Output Rating Frequency (Hz)
4,13.6,AC Output Rating Current (A)
5,3000,AC Output Rating Appearent Power (W)
6,3000,AC Output Rating Active Power (W)
7,24.0,Battery Rating Voltage (V)
8,22.0,Battery Re-charge Voltage (V)
9,20.0,Battery Under Voltage (V)


### Query General Status Parameters

In [16]:
qpigs_data = execute_command('QPIGS').split(' ')

In [17]:
pd.DataFrame(zip(qpigs_data, [
    'AC Voltage (V)',
    'AC Frequency (Hz)',
    'Output Voltage (V)',
    'Output Frequency (Hz)',
    'Output Apparent Power (W)',
    'Output Active Power (W)',
    'Output Load (%)',
    'Bus Voltage (V)',
    'Battery Voltage (V)',
    'Charging Current (A)',
    'Battery Capacity (%)',
    'Heatsink Temperature (°C)',
    'PV Input Current (A)',
    'PV Input Voltage (V)',
    'Battery Voltage From SCC (V)',
    'Battery Discharge Current (A)',
    'Device Status (Flags)',
    'Battery Voltage Offest For Fans On (10mV)',
    'EEPROM Version',
    'PV Charging Power (W)',
    'Device Flags (Flags)',
]), columns=['Value', 'Description'])

,Value,Description
0,217.1,AC Voltage (V)
1,49.5,AC Frequency (Hz)
2,217.1,Output Voltage (V)
3,49.5,Output Frequency (Hz)
4,0183,Output Apparent Power (W)
5,0149,Output Active Power (W)
6,005,Output Load (%)
7,429,Bus Voltage (V)
8,27.00,Battery Voltage (V)
9,007,Charging Current (A)


In [18]:
pd.DataFrame(zip([
    'SBU Priority Version',
    'Configuration Changed',
    'Solar Firmware Version Changed',
    'Has Load',
    'Steady Battery Voltage While Charging',
    'Charge On',
    'Solar Charge On',
    'Utility Charge On',
], qpigs_data[16]), columns=['Flag', 'Value'])

,Flag,Value
0,SBU Priority Version,0
1,Configuration Changed,0
2,Solar Firmware Version Changed,0
3,Has Load,1
4,Steady Battery Voltage While Charging,0
5,Charge On,1
6,Solar Charge On,1
7,Utility Charge On,1


In [19]:
pd.DataFrame(zip([
    'Charging to Floating Mode',
    'Switch On',
    'Dustproof Installed',
], qpigs_data[20]), columns=['Flag', 'Value'])

,Flag,Value
0,Charging to Floating Mode,1
1,Switch On,1
2,Dustproof Installed,0


### Query Default Setting Value Information

In [20]:
qdi_data = execute_command('QDI').split(' ')

In [21]:
pd.DataFrame(zip(qdi_data, [
    'AC Output Voltage (V)',
    'AC Output Frequency (Hz)',
    'Maximum AC Charging Current (A)',
    'Battery Under Voltage (V)',
    'Charging Float Voltage (V)',
    'Charging Bulk Voltage (V)',
    'Battery Default Re-charge Voltage (V)',
    'Max Charging Current (A)',
    'Input Voltage Range (0: Appliance, 1: UPS)',
    'Output Source (0: Utility, 1: Solar)',
    'Charge Source (1: Solar, 2: Solar & Utility, 3: Only Solar)',
    'Battery Type',
    'Silence Buzzer (0/1)',
    'Power Saving (0/1)',
    'Overload Restart (0/1)',
    'Over Temperature Restart (0/1)',
    'LCD Backlight On (0/1)',
    'Alarm On Primary Source Interrupt (0/1)',
    'Fault Code Record (0/1)',
    'Overload Bypass (0/1)',
    'LCD Timeout To Default Page (0/1)',
    'Output Mode (0-4)',
    'Battery Re-discharge Voltage (V)',
    'PK OK (0/1)',
    'PK Power Balance (0/1)',
]), columns=['Value', 'Description'])

,Value,Description
0,230.0,AC Output Voltage (V)
1,50.0,AC Output Frequency (Hz)
2,0030,Maximum AC Charging Current (A)
3,21.0,Battery Under Voltage (V)
4,27.0,Charging Float Voltage (V)
5,28.2,Charging Bulk Voltage (V)
6,23.0,Battery Default Re-charge Voltage (V)
7,50,Max Charging Current (A)
8,0,"Input Voltage Range (0: Appliance, 1: UPS)"
9,0,"Output Source (0: Utility, 1: Solar)"


### Query Device Warning Status

_Details are available in the official documents_

In [22]:
execute_command('QPIWS')

'0000000000000000000000000000000'

### Query Device Flag Status

_Details are available in the official documents_

| Flag | Description                       |
|-----:|:----------------------------------|
|    a | Silence buzzer                    |
|    b | Overload bypass                   |
|    j | Power saving                      |
|    u | Overload retart                   |
|    v | Over temperature restart          |
|    k | Return to default page after 1min |
|    x | Backlight on                      |
|    y | Alarm on primary source interrupt |
|    z | Fault code record                 |

In [23]:
execute_command('QFLAG')

'EkxyzDabjuv'

### Query Battery Equalization Status

In [24]:
qbeqi_data = execute_command('QBEQI').split(' ')

In [25]:
pd.DataFrame(zip(qbeqi_data, [
    'Equalization Enabled (0/1)',
    'Equalization Time (Minute)',
    'Equalization Period (Day)',
    'Equalization Max Current (A)',
    '-Reserved-',
    'Equalization Voltage (V)',
    '-Reserved-',
    'Equalization Over Time (Minute)',
    'Equalization Active Status (0/1)',
    'Equalization Elapse Time (Hour)',
]), columns=['Value', 'Description'])

,Value,Description
0,0,Equalization Enabled (0/1)
1,060,Equalization Time (Minute)
2,030,Equalization Period (Day)
3,060,Equalization Max Current (A)
4,030,-Reserved-
5,29.20,Equalization Voltage (V)
6,000,-Reserved-
7,120,Equalization Over Time (Minute)
8,0,Equalization Active Status (0/1)
9,0000,Equalization Elapse Time (Hour)


In [26]:
connection.close()